## Tensorflow Object Detection Setup For Tensorflow 2.0

In [ ]:
%reset -f

In [ ]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [ ]:
import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
%matplotlib inline

# Tensorflow Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Conv2DTranspose
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
model_dir = r'C:\Users\timot\OneDrive - Johns Hopkins\hackathonProject\models\research\hackathonModel.h5'

## Below is the training data, ignore when running the actual test

In [ ]:
DATA_DIR = r'C:\Users\timot\OneDrive - Johns Hopkins\hackathonProject\models\research\dataset-resized'

train = ImageDataGenerator(rescale= 1/255, validation_split=0.2)
validation = ImageDataGenerator(rescale= 1/255, validation_split=0.2)
train_dataset = train.flow_from_directory(DATA_DIR, target_size=(224,224), batch_size = 32, class_mode = 'categorical', subset= 'training')
validation_dataset = validation.flow_from_directory(DATA_DIR, target_size=(224,224), batch_size = 32, class_mode = 'categorical', subset= 'validation')

In [ ]:
data_augmentation = Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
base_model = tf.keras.applications.VGG19(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False
)


base_model.trainable = False
inputs = tf.keras.Input(shape=(224,224, 3))

x = data_augmentation(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)

outputs = tf.keras.layers.Dense(6, activation='softmax')(x)

model = model = tf.keras.Model(inputs, outputs)

In [ ]:
x_batch, y_batch = next(train_dataset)
print(x_batch.shape)  # Should print (32, 512, 384, 3) based on your input size and batch size
print(y_batch.shape)

In [ ]:
base_model.trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_dataset, validation_data=validation_dataset, epochs=20)

fine tuning

In [ ]:
base_model.trainable = True
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), metrics=['accuracy'])
history_fine = model.fit(train_dataset, epochs=10, validation_data=validation_dataset)

## End of training data

Run this cell if the model is already trained. If just finished training the model, run the save cell before running this

In [ ]:
model.save(model_dir)

In [ ]:
model = tf.keras.models.load_model(model_dir)
print(model_dir)

Perdicting a value

In [ ]:
test_dir = r'C:\Users\timot\OneDrive - Johns Hopkins\hackathonProject\models\research\imagedata'
classarr = ["Cardboard","Glass","Metal","Paper","Plastic","Trash"]

n = 0
for i in os.listdir(test_dir):
    thisImage = image.load_img(test_dir + '//' + i, target_size=(224,224))
    img_array = image.img_to_array(thisImage)
    img_ready = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_ready)
    predicted_class = np.argmax(predictions[0])
    
    if classarr[predicted_class] == "Metal":
        print(f"Predicted class: Metal, item: {i}")
    elif classarr[predicted_class] == "Paper" or classarr[predicted_class] == "Cardboard":
        print(f"Predicted class: Paper or Cardboard, item: {i}")
    elif classarr[predicted_class] == "Plastic" or classarr[predicted_class] == "Glass":
        print(f"Predicted class: Plastic or Glass, item: {i}")
    else:
        print(f"Predicted class: Trash, item: {i}")
    
print(n)


# Detect Objects with OpenCV


## Create the data directory


In [ ]:
import os

DATA_DIR = os.path.join(os.getcwd(), 'data')
MODELS_DIR = os.path.join(DATA_DIR, 'models')
for dir in [DATA_DIR, MODELS_DIR]:
    if not os.path.exists(dir):
        os.mkdir(dir)

## Model

After testing we decided that centernet_resnet50_v2_512x512_coco17_tpu-8 is the best option


In [ ]:
import tarfile
import urllib.request

# Download and extract model
MODEL_DATE = '20200711'
MODEL_NAME = 'centernet_resnet50_v2_512x512_coco17_tpu-8'
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME
print(MODEL_DOWNLOAD_LINK)
PATH_TO_MODEL_TAR = os.path.join(MODELS_DIR, MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))
if not os.path.exists(PATH_TO_CKPT):
    print('Downloading model. This may take a while... ', end='')
    urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
    tar_file = tarfile.open(PATH_TO_MODEL_TAR)
    tar_file.extractall(MODELS_DIR)
    tar_file.close()
    os.remove(PATH_TO_MODEL_TAR)
    print('Done')

# Download labels file
LABEL_FILENAME = 'mscoco_label_map.pbtxt'
LABELS_DOWNLOAD_BASE = \
    'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
PATH_TO_LABELS = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, LABEL_FILENAME))
if not os.path.exists(PATH_TO_LABELS):
    print('Downloading label file... ', end='')
    urllib.request.urlretrieve(LABELS_DOWNLOAD_BASE + LABEL_FILENAME, PATH_TO_LABELS)
    print('Done')

## Load the model
Next we load the downloaded model



In [ ]:

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# # Enable GPU dynamic memory allocation
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

## Load label map data



In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

## Putting everything together


In [ ]:
%%capture
from PIL import UnidentifiedImageError
from PIL import ImageFile
import numpy as np
import pandas as pd
import cv2
import os


import requests

ImageFile.LOAD_TRUNCATED_IMAGES = True

outputPredicted = False
Threshold = 0
    
while True:
#     for i in os.listdir(test_dir):
#         if i == '0.png':
#             try:
#                 os.remove(test_dir + '/0.png')
#             except PermissionError:
#                 print(f"Permission denied for {item_path}.")
#             except Exception as e:
#                 print(f"Error removing {item_path}: {e}")
                
            
    for i in os.listdir(test_dir):
        try:
            thisImage = image.load_img(test_dir + '/' + i, target_size=(224,224))
        except UnidentifiedImageError:
            print(f"Processing file: {test_dir + '/' + i}")
            print(f"Unrecognized image format for file: {i}")
            continue

        image_np_with_detections = img_array.astype('uint8')
        
        #expiremental
        try:
            img = Image.open(test_dir + '/' + i)
            img.verify()  # Check if the image is valid
        except:
            f = 0
#             print("The image might be corrupted or incomplete.")
            
        cvData = cv2.imread(test_dir + '/' + i)
#         print(cvData.shape)

        thisImage = image.load_img(test_dir + '/' + i, target_size=(224,224))
        img_array = image.img_to_array(thisImage)
        
        image_np_with_detections = cv2.cvtColor(image_np_with_detections, cv2.COLOR_RGB2BGR)
        input_tensor = tf.convert_to_tensor(np.expand_dims(img_array, 0), dtype=tf.float32)
        detections, predictions_dict, shapes = detect_fn(input_tensor)
        label_id_offset = 1
        
            
        viz_utils.visualize_boxes_and_labels_on_image_array(
              image_np_with_detections,
              detections['detection_boxes'][0].numpy(),
              (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
              detections['detection_scores'][0].numpy(),
              category_index,
              use_normalized_coordinates=True,
              max_boxes_to_draw=200,
              min_score_thresh=0.4,
              agnostic_mode=False)

#         Display output

        cv2.imshow('object detection', cv2.resize(image_np_with_detections,(800,600)))
    
        
        
#         cv2.imshow("a",cv2.resize(cvData,(800,600)))
#         cv2.waitKey(15)
        
#         person_indices = np.where((detection_classes == PERSON_CLASS_ID) & (detection_scores > THRESHOLD))
#         if person_indices > 0:
    
        thisImage = image.load_img(test_dir + '/' + i, target_size=(224,224))
        img_array = image.img_to_array(thisImage)
        image_np_expanded = np.expand_dims(img_array, axis=0)
        predictions = model.predict(image_np_expanded)
        predicted_class = np.argmax(predictions[0])
        
        

        
        
        
#         val = "Yellow Compost Bin"
#         if classarr[predicted_class] == "Cardboard" or classarr[predicted_class] == "Paper":
#             val = "Blue Paper Recycling Bin"
#         elif classarr[predicted_class] == "Plastic" or classarr[predicted_class] == "Glass":
#             val = "Green Bottle Recycling Bin"
#         elif classarr[predicted_class] == "Metal":
#             val = "Gray Incinerate Bin"
            
        if outputPredicted == False:
            predictions = model.predict(image_np_expanded)
            print(f"Predicted class: {classarr[predicted_class]} item: {i}")
            data = {
                'predicted_class': {classarr[predicted_class]},
                'is_tf': "True"
            }
            r = requests.post(url="http://10.203.127.124:5000/return_prompt",data=data)
            outputPredicted = True
        
        if Threshold >= 10:
            os.remove(test_dir + '/' + i)
            Threshold = 0
            outputPredicted = False
        Threshold = Threshold + 1
#         plt.imshow(img_array/255)
#         plt.show()

    
#Classification


#This function removes the current item



#         if threshold >= 100:
#             threshold = 0
#             for i in os.listdir(test_dir):
#                 item_path = os.path.join(test_dir, i)
#                 if os.path.isfile(item_path):
#                     try:
#                         os.remove(item_path)
#                     except PermissionError:
#                         print(f"Permission denied for {item_path}.")
#                     except Exception as e:
#                         print(f"Error removing {item_path}: {e}")
#             outPredicted = False
    
 
            
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break


#             data = {
#                 'predicted_class': classarr[predicted_class]
#                 'item': i
#             }
#             r = requests.post(url="http://10.203.127.124:5000/return_prompt",data=data)
            
    
#     # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
#     image_np_expanded = np.expand_dims(image_np, axis=0)

#     # Things to try:
#     # Flip horizontally
#     # image_np = np.fliplr(image_np).copy()

#     # Convert image to grayscale
#     # image_np = np.tile(
#     #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

#     input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
#     detections, predictions_dict, shapes = detect_fn(input_tensor)
#     label_id_offset = 1
#     image_np_with_detections = image_np.copy()
    
#     viz_utils.visualize_boxes_and_labels_on_image_array(
#           image_np_with_detections,
#           detections['detection_boxes'][0].numpy(),
#           (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
#           detections['detection_scores'][0].numpy(),
#           category_index,
#           use_normalized_coordinates=True,
#           max_boxes_to_draw=200,
#           min_score_thresh=0.6,
#           agnostic_mode=False)

#     # Display output
#     cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))



cv2.destroyAllWindows()

## All cells below are for testing/debugging purposes

In [ ]:
import numpy as np
import cv2

# Using OpenCV to initialize the webcam
cap = cv2.VideoCapture('video.mp4')



while cap.isOpened():
    ret, image_np = cap.read()
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'][0].numpy(),
          (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
          detections['detection_scores'][0].numpy(),
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.50,
          line_thickness=1,
          agnostic_mode=False)

    cv2.imshow('ssd_mobilenet', image_np_with_detections)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
            
# Release camera and close windows
cap.release()
cv2.destroyAllWindows()  

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture('video.mp4')

while(cap.isOpened()):
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()